# PMOD Digital Pulse Generation

In [ ]:
# Import the QICK drivers and auxiliary libraries
from qick import *
%pylab inline

In [ ]:
# Pyro to communicate with board
import Pyro4
from qick import QickConfig
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99"
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

In [ ]:
# Class containg init method to process pulse sequences for each channel
class DigitalOutput:
    sequence = []

    def __init__(self, pmod_ch, seq):
        self.pmod_ch = pmod_ch
        self.seq = seq

        # Remove pre-exisiting sequence if it already exists for channel
        for l in self.sequence[:]:
            if l[-1] == self.pmod_ch:
                self.sequence.remove(l)
        
        if self.seq != None:
            # Convert list of tuples into list of lists
            self.seq = [list(t) for t in self.seq]

            # Convert pulse widths into elapsed time and append channel to list
            time = 0
            for l in self.seq:
                l[0], time = time, time + l[0]
                l.append(pmod_ch)
            
            # End final pulse
            self.seq.append([time, 0, pmod_ch])

            # Add channel sequence to master sequence and sort in order of time
            [self.sequence.append(l) for l in self.seq]
            self.sequence.sort(key=lambda x: x[0])

In [ ]:
# TODO: Tidy up code block

class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for ch in cfg["ro_chs"]:
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
        phase = self.deg2reg(cfg["res_phase"], gen_ch=res_ch)
        gain = cfg["pulse_gain"]
        self.default_pulse_registers(ch=res_ch, freq=freq, phase=phase, gain=gain)

        style=self.cfg["pulse_style"]

        if style in ["flat_top","arb"]:
            sigma = cfg["sigma"]
            self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)
            
        if style == "const":
            self.set_pulse_registers(ch=res_ch, style=style, length=cfg["length"])
        elif style == "flat_top":
            # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure", length=cfg["length"])
        elif style == "arb":
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure")
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # fire the pulse
        # trigger all declared ADCs
        # pulse PMOD0_0 for a scope trigger
        # pause the tProc until readout is done
        # increment the time counter to give some time before the next measurement
        # (the syncdelay also lets the tProc get back ahead of the clock)
        # self.measure(pulse_ch=self.cfg["res_ch"], 
        #              adcs=self.ro_chs,
        #              adc_trig_offset=self.cfg["adc_trig_offset"],
        #              wait=False,)

        # equivalent to the following:
        self.trigger(adcs=self.ro_chs,
                     adc_trig_offset=self.cfg["adc_trig_offset"])
        self.pulse(ch=self.cfg["res_ch"])

        # AbsArbSignalGen.configure(1, None)

        out = 0
        # trig_output = self.soccfg['tprocs'][0]['trig_output']

        for l in DigitalOutput.sequence: 
            time = int(self.us2cycles(l[0]) / 1e6)
            state = l[1]
            bit_position = l[2]

            if state == 1:
                out |= (1 << bit_position)
            elif state == 0:
                out &= ~(1 << bit_position)

            print(bin(out), time)
            self.regwi(0, 31, out)
            self.seti(7, 0, 31, time) # TODO - arguments are hacky

        self.wait_all()
        self.sync_all(self.us2cycles(self.cfg["relax_delay"]))

In [ ]:
# Program configuration
config={"res_ch":1, # --Fixed
        "ro_chs":[1], # --Fixed
        "reps":1, # TODO - Understand behaviour
        "relax_delay":1.0, # --us
        "res_phase":0, # --degrees
        "pulse_style": "const", # --Fixed
        "length":20, # [Clock ticks] Try varying length from 10-100 clock ticks 
        "readout_length":200, # [Clock ticks] Try varying readout_length from 50-1000 clock ticks
        "pulse_gain":3000, # [DAC units] Min:500, Max: 30000, Gain of ADC output
        "pulse_freq": 0, # [MHz] Frequency of ADC output oscillations
        "adc_trig_offset": 0, # [Clock ticks], ADC time offset
        "soft_avgs":100 # Number of repeats program executes and averages
       }

In [ ]:
# Declare sequences of pulses.
# PMOD channel (int) is first argument
# Pulses are in list of tuples, where 0th index contains pulse width (int), and 1st index contains pulse state (bool).
seq_0 = DigitalOutput(0, [(100e3, 1), (150e3, 0), (250e3, 1), (100e3, 0), (150e3, 1)])

# seq_1 = DigitalOutput(1, [(100, 0), (150, 1), (250, 0), (100, 1)])
# seq_3 = DigitalOutput(0, None)
# print(DigitalOutput.sequence)

In [ ]:
# Run program
prog = LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, progress=False)